In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
data_path = "./data/origin_"
glob.glob("./data/origin*")

['./data/origin_gensim_summarization.csv',
 './data/origin_product.csv',
 './data/origin_textrank_lexrank.csv',
 './data/origin_user.csv',
 './data/origin_purchase.csv']

# 1. product 전처리

In [4]:
product_origin = pd.read_csv(data_path+"product.csv")
print(product_origin.columns)
product_origin.tail()


Index(['goodsno', 'url', '상품명', '카테고리', '브랜드', '가격', '용량', '주요사양', '성분',
       '전체 평점', '5점', '4점', '3점', '2점', '1점', '1_1피부', '1_2피부', '1_3피부',
       '2_1피부', '2_2피부', '2_3피부', '3_1피부', '3_2피부', '3_3피부', '증정 여부'],
      dtype='object')


,goodsno,url,상품명,카테고리,브랜드,가격,용량,주요사양,성분,전체 평점,...,1_1피부,1_2피부,1_3피부,2_1피부,2_2피부,2_3피부,3_1피부,3_2피부,3_3피부,증정 여부
463,A000000184579,https://www.oliveyoung.co.kr/store/goods/getGo...,[NEW] 폴메디슨 옴므 워터화이트닝 올인원 에센스 211ml,올인원,폴메디슨,"13,800",211ml,모든피부,"정제수, 다이프로필렌글라이콜, 병풀추출물, 파스향나무잎추출물, 에탄올, 세틸에틸헥사...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
464,A000000184652,https://www.oliveyoung.co.kr/store/goods/getGo...,아이오페 스템3 소프너 150ml,스킨/토너,아이오페,"33,600",150ML,핵심 효능 성분 알란토 컴플렉스가 약해진 피부 장벽을 개선 하고 지친 피부 컨디션을...,"정제수 · 부틸렌글라이콜 · 글리세린 · 베타인 · 나이아신아마이드 · 1,2-헥산...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
465,A000000011084,https://www.oliveyoung.co.kr/store/goods/getGo...,엠도씨 릴리프 토닉 위드 애프터셰이브 150ml,스킨/토너,엠도씨,"20,000",150ml,모든 피부용,"정제수,사이클로펜타실록세인,글리세린,네오펜틸글라이콜다이헵타노에이트,메틸프로판다이올,...",4.6,...,22.0,64.0,14.0,33.0,67.0,1.0,43.0,55.0,2.0,0.0
466,A000000184664,https://www.oliveyoung.co.kr/store/goods/getGo...,아이오페 맨 안티에이징 에멀젼EX 120ml,로션/에멀젼,아이오페,"36,000",120ML.,피부 주름을 개선하고 탄력을 강화해주는 안티에이징 보습 에멀젼,"정제수, 부틸렌글라이콜, 세틸에틸헥사노에이트, 글리세린, 펜타에리스리틸테트라에틸헥사...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
467,A000000182369,https://www.oliveyoung.co.kr/store/goods/getGo...,[NEW] 닥터지 레드 블레미쉬 클리어 수딩 토너 300ml+300ml 듀오기획,스킨/토너,닥터지,"39,900",300ml+300ml,모든 피부 / 민감 피부,"정제수, 다이프로필렌글라이콜, 프로판다이올, 펜틸렌글라이콜, 나이아신아마이드, 부틸...",4.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## 1-1 column 이름 영어로

In [5]:
product_origin.columns = [
  'no',
  'url',
  'pid',
  'category',
  'brand',
  'price',
  'volume',
  'spec',
  'ingredient',
  'rate_total',
  'rate_5',
  'rate_4',
  'rate_3',
  'rate_2',
  'rate_1',
  'type_11',
  'type_12',
  'type_13',
  'type_21',
  'type_22',
  'type_23',
  'type_31',
  'type_32',
  'type_33',
  'present']

product = product_origin.drop(['no', 'url'], axis = 1).reset_index()
product

,index,pid,category,brand,price,volume,spec,ingredient,rate_total,rate_5,...,type_11,type_12,type_13,type_21,type_22,type_23,type_31,type_32,type_33,present
0,0,우르오스 스킨로션 200ml,올인원,우르오스,"23,700",200ml,지복합성 피부를 위한 워터타입 스킨케어 [ 2 in 1 ],"정제수, 에탄올, 펜틸렌글라이콜, 글리세린, 베타인, 피이지-6, 피이지-32, 아...",4.7,81.0,...,16.0,59.0,25.0,57.0,41.0,2.0,57.0,41.0,2.0,0.0
1,1,우르오스 스킨밀크 200ml,올인원,우르오스,"23,700",200ml,중건성 피부를 위한 밀크타입 스킨케어 [ 2 in 1 ],"정제수, 글리세린, 하이드로제네이티드폴리(C6-12올레핀), 에탄올, 폴리글리세릴-...",4.8,80.0,...,35.0,60.0,5.0,74.0,26.0,1.0,64.0,36.0,1.0,0.0
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),스킨/토너,라운드랩,"27,000",[본품]라운드랩 1025 독도 토너 500ml+[증정품]라운드랩 1025 독도 토너...,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",4.8,86.0,...,22.0,63.0,15.0,39.0,61.0,1.0,80.0,20.0,1.0,1.0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,로션/에멀젼,에스트라,"24,800",150ml,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 스쿠알란,펜타에리스리틸테트라이소스테아레이트, 사...",4.8,85.0,...,46.0,49.0,5.0,70.0,30.0,0.0,73.0,27.0,0.0,1.0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,스킨/토너,토리든,"15,700",[본품] 다이브인 토너 300ml [증정] 다이브인 토너 100ml,모든 피부 타입,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...",4.8,83.0,...,25.0,57.0,18.0,52.0,48.0,1.0,77.0,23.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,463,[NEW] 폴메디슨 옴므 워터화이트닝 올인원 에센스 211ml,올인원,폴메디슨,"13,800",211ml,모든피부,"정제수, 다이프로필렌글라이콜, 병풀추출물, 파스향나무잎추출물, 에탄올, 세틸에틸헥사...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
464,464,아이오페 스템3 소프너 150ml,스킨/토너,아이오페,"33,600",150ML,핵심 효능 성분 알란토 컴플렉스가 약해진 피부 장벽을 개선 하고 지친 피부 컨디션을...,"정제수 · 부틸렌글라이콜 · 글리세린 · 베타인 · 나이아신아마이드 · 1,2-헥산...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
465,465,엠도씨 릴리프 토닉 위드 애프터셰이브 150ml,스킨/토너,엠도씨,"20,000",150ml,모든 피부용,"정제수,사이클로펜타실록세인,글리세린,네오펜틸글라이콜다이헵타노에이트,메틸프로판다이올,...",4.6,73.0,...,22.0,64.0,14.0,33.0,67.0,1.0,43.0,55.0,2.0,0.0
466,466,아이오페 맨 안티에이징 에멀젼EX 120ml,로션/에멀젼,아이오페,"36,000",120ML.,피부 주름을 개선하고 탄력을 강화해주는 안티에이징 보습 에멀젼,"정제수, 부틸렌글라이콜, 세틸에틸헥사노에이트, 글리세린, 펜타에리스리틸테트라에틸헥사...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [6]:
num2pid = product.pid.to_dict()
pid2num = {val:key for key, val in num2pid.items()}

## 1-2 category 컬럼을 one-hot encoding

In [7]:
category_df = pd.get_dummies(product.category)
num2category = {key: val for key,val in enumerate(category_df.columns)}
category_df.columns = ["category_{}".format(i) for i in range(len(category_df.columns))]
product = pd.concat([product, category_df], axis = 1)
product = product.drop(["category"], axis = 1)
product.head()

,index,pid,brand,price,volume,spec,ingredient,rate_total,rate_5,rate_4,...,type_22,type_23,type_31,type_32,type_33,present,category_0,category_1,category_2,category_3
0,0,우르오스 스킨로션 200ml,우르오스,"23,700",200ml,지복합성 피부를 위한 워터타입 스킨케어 [ 2 in 1 ],"정제수, 에탄올, 펜틸렌글라이콜, 글리세린, 베타인, 피이지-6, 피이지-32, 아...",4.7,81.0,14.0,...,41.0,2.0,57.0,41.0,2.0,0.0,0,0,0,1
1,1,우르오스 스킨밀크 200ml,우르오스,"23,700",200ml,중건성 피부를 위한 밀크타입 스킨케어 [ 2 in 1 ],"정제수, 글리세린, 하이드로제네이티드폴리(C6-12올레핀), 에탄올, 폴리글리세릴-...",4.8,80.0,16.0,...,26.0,1.0,64.0,36.0,1.0,0.0,0,0,0,1
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),라운드랩,"27,000",[본품]라운드랩 1025 독도 토너 500ml+[증정품]라운드랩 1025 독도 토너...,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",4.8,86.0,11.0,...,61.0,1.0,80.0,20.0,1.0,1.0,0,1,0,0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,에스트라,"24,800",150ml,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 스쿠알란,펜타에리스리틸테트라이소스테아레이트, 사...",4.8,85.0,12.0,...,30.0,0.0,73.0,27.0,0.0,1.0,1,0,0,0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,토리든,"15,700",[본품] 다이브인 토너 300ml [증정] 다이브인 토너 100ml,모든 피부 타입,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...",4.8,83.0,12.0,...,48.0,1.0,77.0,23.0,1.0,1.0,0,1,0,0


In [8]:
product.price = product.price.apply(lambda x:''.join(x.split(','))).astype('int')
product.head()

,index,pid,brand,price,volume,spec,ingredient,rate_total,rate_5,rate_4,...,type_22,type_23,type_31,type_32,type_33,present,category_0,category_1,category_2,category_3
0,0,우르오스 스킨로션 200ml,우르오스,23700,200ml,지복합성 피부를 위한 워터타입 스킨케어 [ 2 in 1 ],"정제수, 에탄올, 펜틸렌글라이콜, 글리세린, 베타인, 피이지-6, 피이지-32, 아...",4.7,81.0,14.0,...,41.0,2.0,57.0,41.0,2.0,0.0,0,0,0,1
1,1,우르오스 스킨밀크 200ml,우르오스,23700,200ml,중건성 피부를 위한 밀크타입 스킨케어 [ 2 in 1 ],"정제수, 글리세린, 하이드로제네이티드폴리(C6-12올레핀), 에탄올, 폴리글리세릴-...",4.8,80.0,16.0,...,26.0,1.0,64.0,36.0,1.0,0.0,0,0,0,1
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),라운드랩,27000,[본품]라운드랩 1025 독도 토너 500ml+[증정품]라운드랩 1025 독도 토너...,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",4.8,86.0,11.0,...,61.0,1.0,80.0,20.0,1.0,1.0,0,1,0,0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,에스트라,24800,150ml,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 스쿠알란,펜타에리스리틸테트라이소스테아레이트, 사...",4.8,85.0,12.0,...,30.0,0.0,73.0,27.0,0.0,1.0,1,0,0,0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,토리든,15700,[본품] 다이브인 토너 300ml [증정] 다이브인 토너 100ml,모든 피부 타입,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...",4.8,83.0,12.0,...,48.0,1.0,77.0,23.0,1.0,1.0,0,1,0,0


## 1-3 volume 전처리

In [9]:
import re

def find_volume(sentence):
	result = re.findall(r'\d+ml', sentence)
	if len(result) == 0: result = re.findall(r'\d+ml', sentence)
	elif len(result) == 0: result = re.findall(r'\d+ ml', sentence)
	elif len(result) == 0: result = re.findall(r'\d+ML', sentence)
	elif len(result) == 0: result = re.findall(r'\d+ml', sentence)
	elif len(result) == 0: return 0

	result = [re.findall(r'\d+', i)[0] for i in result]
	if len(result) == 1:
		return int(result[0])
	else:
		return sum([int(i) for i in result])

In [9]:
product.volume = product.volume.apply(find_volume)
product.head()

,index,pid,brand,price,volume,spec,ingredient,rate_total,rate_5,rate_4,...,type_22,type_23,type_31,type_32,type_33,present,category_0,category_1,category_2,category_3
0,0,우르오스 스킨로션 200ml,우르오스,23700,200,지복합성 피부를 위한 워터타입 스킨케어 [ 2 in 1 ],"정제수, 에탄올, 펜틸렌글라이콜, 글리세린, 베타인, 피이지-6, 피이지-32, 아...",4.7,81.0,14.0,...,41.0,2.0,57.0,41.0,2.0,0.0,0,0,0,1
1,1,우르오스 스킨밀크 200ml,우르오스,23700,200,중건성 피부를 위한 밀크타입 스킨케어 [ 2 in 1 ],"정제수, 글리세린, 하이드로제네이티드폴리(C6-12올레핀), 에탄올, 폴리글리세릴-...",4.8,80.0,16.0,...,26.0,1.0,64.0,36.0,1.0,0.0,0,0,0,1
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),라운드랩,27000,600,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",4.8,86.0,11.0,...,61.0,1.0,80.0,20.0,1.0,1.0,0,1,0,0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,에스트라,24800,150,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 스쿠알란,펜타에리스리틸테트라이소스테아레이트, 사...",4.8,85.0,12.0,...,30.0,0.0,73.0,27.0,0.0,1.0,1,0,0,0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,토리든,15700,400,모든 피부 타입,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...",4.8,83.0,12.0,...,48.0,1.0,77.0,23.0,1.0,1.0,0,1,0,0


## 1-4 spec 전처리

In [10]:
def find_spec(sentence):
	if "건성" in sentence or "건조" in sentence: return '건성'
	if "지성" in sentence: return '지성'
	if "복합" in sentence: return '복합'
	if "민감" in sentence: return '민감'
	if "트러블" in sentence: return '트러블'
	if "남성" in sentence or "쉐이빙" in sentence or  "쉐이브" in sentence: return '남성'
	# if "모든" in sentence or "보통" in sentence or "모듣" in sentence: return 6
	else: return '모든'

In [11]:
spec_df = product.spec.apply(find_spec)
spec_df = pd.get_dummies(spec_df)
num2spec = {key: val for key,val in enumerate(spec_df.columns)}
spec_df.columns = ["spec_{}".format(i) for i in range(len(spec_df.columns))]
product = pd.concat([product, spec_df], axis = 1)
product = product.drop(["spec"], axis = 1)
product.head()

,index,pid,brand,price,volume,ingredient,rate_total,rate_5,rate_4,rate_3,...,category_1,category_2,category_3,spec_0,spec_1,spec_2,spec_3,spec_4,spec_5,spec_6
0,0,우르오스 스킨로션 200ml,우르오스,23700,200,"정제수, 에탄올, 펜틸렌글라이콜, 글리세린, 베타인, 피이지-6, 피이지-32, 아...",4.7,81.0,14.0,4.0,...,0,0,1,0,0,0,0,1,0,0
1,1,우르오스 스킨밀크 200ml,우르오스,23700,200,"정제수, 글리세린, 하이드로제네이티드폴리(C6-12올레핀), 에탄올, 폴리글리세릴-...",4.8,80.0,16.0,3.0,...,0,0,1,1,0,0,0,0,0,0
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),라운드랩,27000,600,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",4.8,86.0,11.0,2.0,...,1,0,0,0,0,1,0,0,0,0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,에스트라,24800,150,"정제수, 부틸렌글라이콜, 글리세린, 스쿠알란,펜타에리스리틸테트라이소스테아레이트, 사...",4.8,85.0,12.0,3.0,...,0,0,0,0,0,1,0,0,0,0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,토리든,15700,400,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...",4.8,83.0,12.0,3.0,...,1,0,0,0,0,1,0,0,0,0


## 1-5 Ingredient 전처리

In [12]:
# ingredient 추출 및 등장횟수 카운팅
# count가 threshold 보다 작다면 무시
ingredient_df = product.ingredient.apply(lambda x: x.split(', '))
all_ingredient = []
for ele in ingredient_df:
    all_ingredient = all_ingredient + ele
ingredient_count = dict.fromkeys(set(all_ingredient), 0)

for ele in all_ingredient:
    ingredient_count[ele] += 1
count = 0
thresh = 30
del_key = []
for key in ingredient_count.keys():
	if ingredient_count[key] > thresh:
		print(key, ingredient_count[key])
		count += 1
	else:
		del_key.append(key)

for key in del_key:
	del ingredient_count[key]

#정제수 제외
del ingredient_count['정제수']
count -= 1

print(count)

사이클로펜타실록세인 32
부틸렌글라이콜 352
하이드록시에틸아크릴레이트/소듐아크릴로일다이메틸타우레이트코폴리머 44
병풀추출물 116
카프릴릭/카프릭트라이글리세라이드 93
나이아신아마이드 161
시트릭애씨드 94
폴리글리세릴-10라우레이트 35
하이알루로닉애씨드 103
알지닌 71
에틸헥실글리세린 242
하이드로제네이티드레시틴 144
다이포타슘글리시리제이트 49
스쿠알란 72
하이드록시프로필트라이모늄하이알루로네이트 44
마트리카리아꽃추출물 31
아시아티코사이드 47
잔탄검 149
실리카 54
하이드롤라이즈드소듐하이알루로네이트 33
로즈마리잎오일 34
베타인 146
글루코오스 64
소듐아세틸레이티드하이알루로네이트 58
다이메티콘 52
제라니올 47
글리세릴스테아레이트 85
락토바실러스발효물 32
녹차추출물 65
팔미틱애씨드 49
펜틸렌글라이콜 130
마데카식애씨드 45
소듐하이드록사이드 42
황금추출물 31
암모늄아크릴로일다이메틸타우레이트/브이피코폴리머 45
1,2-헥산다이올 325
당느릅나무뿌리추출물 39
아크릴레이트/C10-30알킬아크릴레이트크로스폴리머 94
소듐하이알루로네이트 209
트레할로오스 47
변성알코올 70
정제수 293
솔비탄올리베이트 31
에탄올 52
아데노신 145
판테놀 189
시트로넬올 60
페녹시에탄올 61
세테아릴올리베이트 32
프로판다이올 169
베르가모트오일 34
마데카소사이드 82
스테아릭애씨드 64
알란토인 139
레시틴 42
메틸프로판다이올 63
덱스트린 34
쇠비름추출물 50
세라마이드엔피 119
소듐하이알루로네이트크로스폴리머 73
하이드로제네이티드폴리(C6-14올레핀) 31
다이소듐이디티에이 220
트로메타민 175
시트랄 34
글리세레스-26 40
글리세릴카프릴레이트 40
베타-글루칸 62
소듐시트레이트 55
하이드록시아세토페논 44
카보머 153
살리실릭애씨드 42
콜레스테롤 42
자일리톨 37
리날룰 101
토코페롤 137
다이프로필렌글라이콜 120
하이드롤라이즈드하이알루로닉애씨드 115
글루코노락톤 44
글리세린 39

In [13]:
# ingredient를 정수로 mapping
ingredient2num = {}
num2ingredient = {}
for i, key in enumerate(ingredient_count.keys()):
	ingredient2num[key] = i
	num2ingredient[i] = key

In [14]:
# ingredient onehot
ingredient_onehot = np.zeros((len(product), count))
for i, row in enumerate(ingredient_df):
	j = []
	for ingredient in row:
		if ingredient in ingredient2num.keys():
			j.append(ingredient2num[ingredient])
	if len(j)> 0:
		ingredient_onehot[i,j] =1


In [15]:
# 각 성분 중 하나도 없는 화장픔도 존재
ingredient_onehot.sum(axis=1)

array([10.,  9., 13.,  8., 16., 17., 37., 18., 43.,  9., 10., 14., 16.,
       13., 24., 25., 31., 27.,  0., 25., 12., 15., 16., 15.,  0.,  7.,
        0., 44., 14.,  0.,  8., 24., 16., 35., 18., 28., 13., 23.,  9.,
       24.,  9., 19.,  2., 19., 19., 19., 11., 19., 24., 25.,  0.,  8.,
       37., 10., 16., 23., 25., 23., 29., 32., 20.,  8.,  8., 23., 12.,
       13.,  0., 20., 30.,  0., 25., 18., 14., 61., 41.,  0., 37., 19.,
       10.,  0., 10.,  0.,  9., 21., 22., 19.,  0., 33., 26., 16., 21.,
       20.,  0.,  0.,  8., 15., 15.,  7., 13.,  0.,  9., 20.,  2., 25.,
       12., 16., 15., 15., 28.,  0., 27.,  3., 29., 18., 15., 19.,  0.,
        0., 34.,  4., 16., 57., 29.,  0., 21., 25., 22., 19., 19.,  0.,
        0., 20., 21., 11., 34., 19., 12.,  9.,  5., 15., 25., 25.,  0.,
        0.,  8., 10., 21., 35., 12.,  8., 26., 16.,  2.,  9., 18., 31.,
       20., 17., 31.,  0., 22., 43., 23.,  0., 32., 19., 20., 23.,  0.,
       15., 18.,  0.,  8., 36.,  0.,  1., 11., 27., 19., 10., 15

In [16]:
ingredient_df = pd.DataFrame(ingredient_onehot, columns=["in_{}".format(i) for i in range(count)])
product = pd.concat([product, ingredient_df], axis = 1)
product = product.drop('ingredient', axis =1)
product.head()

,index,pid,brand,price,volume,rate_total,rate_5,rate_4,rate_3,rate_2,...,in_81,in_82,in_83,in_84,in_85,in_86,in_87,in_88,in_89,in_90
0,0,우르오스 스킨로션 200ml,우르오스,23700,200,4.7,81.0,14.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,우르오스 스킨밀크 200ml,우르오스,23700,200,4.8,80.0,16.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),라운드랩,27000,600,4.8,86.0,11.0,2.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,에스트라,24800,150,4.8,85.0,12.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,토리든,15700,400,4.8,83.0,12.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
brand_count= product.brand.value_counts()
print(sum(brand_count>5))
brand_count = brand_count[brand_count>5]
brand_count

30


닥터지       19
라운드랩      15
클럽맨피노드    13
비오템 옴므    11
피지오겔      11
엠도씨       10
랩시리즈      10
플리프        9
한율         8
크리니크       8
아누아        8
아이디얼포맨     8
디오디너리      8
우르오스       7
아이소이       7
마몽드        7
차앤박옴므      7
아이오페       7
싸이닉        7
케어존        7
라네즈        7
라끄베르       7
하다라보       6
미프         6
비욘드        6
넘버즈인       6
아크네스       6
닥터자르트      6
바이오힐보      6
유리아쥬       6
Name: brand, dtype: int64

In [18]:
brand2num = {}
num2brand = {}
for i, brand in enumerate(brand_count.index):
	brand2num[brand] = i
	num2brand[i] = brand
brand2num

{'닥터지': 0,
 '라운드랩': 1,
 '클럽맨피노드': 2,
 '비오템 옴므': 3,
 '피지오겔': 4,
 '엠도씨': 5,
 '랩시리즈': 6,
 '플리프': 7,
 '한율': 8,
 '크리니크': 9,
 '아누아': 10,
 '아이디얼포맨': 11,
 '디오디너리': 12,
 '우르오스': 13,
 '아이소이': 14,
 '마몽드': 15,
 '차앤박옴므': 16,
 '아이오페': 17,
 '싸이닉': 18,
 '케어존': 19,
 '라네즈': 20,
 '라끄베르': 21,
 '하다라보': 22,
 '미프': 23,
 '비욘드': 24,
 '넘버즈인': 25,
 '아크네스': 26,
 '닥터자르트': 27,
 '바이오힐보': 28,
 '유리아쥬': 29}

In [19]:
brand_df = pd.get_dummies(product.brand.apply(lambda x:brand2num[x] if x in brand2num.keys() else np.nan))
brand_df.columns = ['brand_{}'.format(int(i)) for i in brand_df.columns]
product = pd.concat([product, brand_df], axis = 1)
product = product.drop('brand', axis =1)
product.head()

,index,pid,price,volume,rate_total,rate_5,rate_4,rate_3,rate_2,rate_1,...,brand_20,brand_21,brand_22,brand_23,brand_24,brand_25,brand_26,brand_27,brand_28,brand_29
0,0,우르오스 스킨로션 200ml,23700,200,4.7,81.0,14.0,4.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,우르오스 스킨밀크 200ml,23700,200,4.8,80.0,16.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,[대용량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),27000,600,4.8,86.0,11.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,에스트라 아토베리어365 로션 150ml 기획 (+하이드로에센스 25ml+무기자차선...,24800,150,4.8,85.0,12.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 300ml 기획(+100ml 추...,15700,400,4.8,83.0,12.0,3.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# 2. User 전처리

In [20]:
user = pd.read_csv(data_path+"user.csv")
user.columns  = ['uid', 'type1', 'type2', 'type3', 'type4', 'ranking']
user.head()

,uid,type1,type2,type3,type4,ranking
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,지성,웜톤,모공,트러블,1792
1,ckN0NlJxTy9WU2IyMjZQWHQ3dWpzUT09,건성,웜톤,주름,탄력,0
2,U3Eyb2xHVytUcFVYajdSbkJVaUV3UT09,민감성,겨울쿨톤,각질,다크서클,0
3,T3VnTkYrTnFocnkzdktxVVg3VFJuQT09,복합성,웜톤,주름,탄력,0
4,ekdrWEZHc1dYb2ZJTEpDVXI3Sm1adz09,건성,쿨톤,각질,모공,0


In [21]:
# 공백을 nan으로
user = user.replace('-', np.nan)
user

,uid,type1,type2,type3,type4,ranking
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,지성,웜톤,모공,트러블,1792
1,ckN0NlJxTy9WU2IyMjZQWHQ3dWpzUT09,건성,웜톤,주름,탄력,0
2,U3Eyb2xHVytUcFVYajdSbkJVaUV3UT09,민감성,겨울쿨톤,각질,다크서클,0
3,T3VnTkYrTnFocnkzdktxVVg3VFJuQT09,복합성,웜톤,주름,탄력,0
4,ekdrWEZHc1dYb2ZJTEpDVXI3Sm1adz09,건성,쿨톤,각질,모공,0
...,...,...,...,...,...,...
5404,M1QwNU8wdGdlNGlaaWt2SWZaSHY5QT09,NaN,NaN,NaN,NaN,0
5405,VXBNMzczaFJwQnhVY0ZNc0NYN3JkUT09,NaN,NaN,NaN,NaN,0
5406,c21qbWp5c2w5d2RLdWxqTmsvM2tKdz09,NaN,NaN,NaN,NaN,0
5407,dGpJaThLa2toT1hNQ0ZjTlI3MUdlQT09,NaN,NaN,NaN,NaN,0


## 2-2 type1을 one-hot encoding

In [22]:
type1 = pd.get_dummies(user.type1)
user = pd.concat([user, type1], axis=1)
user = user.drop('type1', axis =1)
user.head()

,uid,type2,type3,type4,ranking,건성,민감성,복합성,약건성,중성,지성,트러블성
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,웜톤,모공,트러블,1792,0,0,0,0,0,1,0
1,ckN0NlJxTy9WU2IyMjZQWHQ3dWpzUT09,웜톤,주름,탄력,0,1,0,0,0,0,0,0
2,U3Eyb2xHVytUcFVYajdSbkJVaUV3UT09,겨울쿨톤,각질,다크서클,0,0,1,0,0,0,0,0
3,T3VnTkYrTnFocnkzdktxVVg3VFJuQT09,웜톤,주름,탄력,0,0,0,1,0,0,0,0
4,ekdrWEZHc1dYb2ZJTEpDVXI3Sm1adz09,쿨톤,각질,모공,0,1,0,0,0,0,0,0


## 2-3 type2을 one-hot encoding

In [23]:
type2 = pd.get_dummies(user.type2)
user = pd.concat([user, type2], axis=1)
user = user.drop('type2', axis =1)
user.head()

,uid,type3,type4,ranking,건성,민감성,복합성,약건성,중성,지성,트러블성,가을웜톤,겨울쿨톤,봄웜톤,여름쿨톤,웜톤,쿨톤
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,모공,트러블,1792,0,0,0,0,0,1,0,0,0,0,0,1,0
1,ckN0NlJxTy9WU2IyMjZQWHQ3dWpzUT09,주름,탄력,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,U3Eyb2xHVytUcFVYajdSbkJVaUV3UT09,각질,다크서클,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,T3VnTkYrTnFocnkzdktxVVg3VFJuQT09,주름,탄력,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,ekdrWEZHc1dYb2ZJTEpDVXI3Sm1adz09,각질,모공,0,1,0,0,0,0,0,0,0,0,0,0,0,1


## 2-4 type3,4 합치고 one-hot encoding

In [24]:
type3 = pd.get_dummies(user.type3)
type3_val = type3.iloc[:,1:].values #각질 제외

type4 = pd.get_dummies(user.type4)
type4_val = type4.values
type34 = type3_val + type4_val
type3.iloc[:,1:] = type34

user = pd.concat([user, type3], axis=1)
user = user.drop(['type3', 'type4'], axis =1)
user.head()

,uid,ranking,건성,민감성,복합성,약건성,중성,지성,트러블성,가을웜톤,...,미백,민감성,블랙헤드,아토피,잡티,주름,탄력,트러블,피지과다,홍조
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,1792,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,ckN0NlJxTy9WU2IyMjZQWHQ3dWpzUT09,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,U3Eyb2xHVytUcFVYajdSbkJVaUV3UT09,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,T3VnTkYrTnFocnkzdktxVVg3VFJuQT09,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,ekdrWEZHc1dYb2ZJTEpDVXI3Sm1adz09,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2-5 ranking 전처리

In [25]:
max_ranking = user.ranking.max() + 1
user.loc[user.ranking != 0, 'ranking'] =  (max_ranking - user.loc[user.ranking != 0, 'ranking'])/(2*max_ranking) + 0.5
user

,uid,ranking,건성,민감성,복합성,약건성,중성,지성,트러블성,가을웜톤,...,미백,민감성,블랙헤드,아토피,잡티,주름,탄력,트러블,피지과다,홍조
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,0.679428,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,ckN0NlJxTy9WU2IyMjZQWHQ3dWpzUT09,0.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,U3Eyb2xHVytUcFVYajdSbkJVaUV3UT09,0.000000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,T3VnTkYrTnFocnkzdktxVVg3VFJuQT09,0.000000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,ekdrWEZHc1dYb2ZJTEpDVXI3Sm1adz09,0.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5404,M1QwNU8wdGdlNGlaaWt2SWZaSHY5QT09,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5405,VXBNMzczaFJwQnhVY0ZNc0NYN3JkUT09,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5406,c21qbWp5c2w5d2RLdWxqTmsvM2tKdz09,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5407,dGpJaThLa2toT1hNQ0ZjTlI3MUdlQT09,0.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 3. purchase 전처리

In [26]:
purchase_origin = pd.read_csv(data_path+"purchase.csv")
purchase_origin.columns = ['uid',"type1", "type2", "type3", "type4", "brand", "pid", "rate", "date", "review", "ranking"]
# purchase_origin.uid = purchase_origin.uid.apply(lambda x: uid2num[x])
purchase = purchase_origin.drop(['type1', 'type2', 'type3', 'type4', 'date', 'ranking' ], axis =1)
purchase.loc[purchase.review.isna(),'review'] = ' '
purchase.loc[purchase.review == np.nan,'review'] = ' '
purchase.head()

,uid,brand,pid,rate,review
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,라운드랩,[NEW] 라운드랩 소나무 진정 시카 마스크 10매,4,제가 참 x랄같은 피부라 화장품 수억 버렸죠..휴..폼하나도 정말신중의 신중을 다해...
1,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,라운드랩,라운드랩 자작나무 수분 마스크 1매,4,이벤트 할때 한장사서.써보고 괜찮아서 다시 구입합니다..저는 지성피부예요..나이 5...
2,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,라운드랩,라운드랩 해풍쑥 진정 마스크 10매,5,해풍쑥이라고 해서 쑥향이나진않아요..지난부 라운드랩이 대대적인 이벤트를.해서 마스크...
3,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,락토핏,락토핏 골드 트리플 기획세트 (3개월분),5,항상 먹는 유산균이예요..할인.이벤트해서 구입했어요..출고를 엄청늦게하심..주문한지...
4,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,락토핏,락토핏 코어 30포 (1개월분),5,항상 락토핏 골드만.먹다가 이번에 이벤트할인 많이 하길래래 구입했습니다..맛은 동일...


In [27]:
purchase.pid = purchase.pid.apply(lambda x: pid2num[x] if x in pid2num.keys() else np.nan)
purchase

,uid,brand,pid,rate,review
0,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,라운드랩,NaN,4,제가 참 x랄같은 피부라 화장품 수억 버렸죠..휴..폼하나도 정말신중의 신중을 다해...
1,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,라운드랩,NaN,4,이벤트 할때 한장사서.써보고 괜찮아서 다시 구입합니다..저는 지성피부예요..나이 5...
2,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,라운드랩,NaN,5,해풍쑥이라고 해서 쑥향이나진않아요..지난부 라운드랩이 대대적인 이벤트를.해서 마스크...
3,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,락토핏,NaN,5,항상 먹는 유산균이예요..할인.이벤트해서 구입했어요..출고를 엄청늦게하심..주문한지...
4,eHNCNmFVY1ladXNQV21xcS9zT0J1QT09,락토핏,NaN,5,항상 락토핏 골드만.먹다가 이번에 이벤트할인 많이 하길래래 구입했습니다..맛은 동일...
...,...,...,...,...,...
89928,Q2pJRjdyY0M4SjhySkpTZWRCL2hBQT09,아로마티카,NaN,5,자극 없고 좋아요! 세일할 때 사서 두병에 정말 저렴하게 샀어요! 재구매 의사 있음
89929,Q2pJRjdyY0M4SjhySkpTZWRCL2hBQT09,랩노쉬,NaN,5,전에 먹었던 쿠키앤 크림보다 블랙쿠키가 더 맛있어요!\n영양성분이 좋은 건 모르겠으...
89930,Q2pJRjdyY0M4SjhySkpTZWRCL2hBQT09,딜라이트 프로젝트,NaN,5,요거트에 넣어 먹고 있습니다 .. 너무 맛있어요 다들 꼭 해드십쇼 ...! 양도 이...
89931,Q2pJRjdyY0M4SjhySkpTZWRCL2hBQT09,아리얼,NaN,5,말해뭐해 피부 화장 지울 때 짱이에요! 부드러워요 다만 워터프루프 제품은 리무버로 ...


In [28]:
purchase = purchase.loc[~purchase.pid.isna()]
purchase = purchase.drop(['brand'], axis = 1)
purchase.pid = purchase.pid.astype('int')

In [29]:
user_review_agg = purchase.loc[:,['uid','review']].groupby('uid')['review'].apply(lambda x: ' '.join(x))
user_review_agg

uid
K000TGNRK0RYTHFlYXlySkFPd3lJdz09    처음 바를때는 따가웠는데\n자고일어나도 유분기 없이 보송하게되어서좋고\n대용량이라 ...
K01rTk5mU3I1WWN5SmcrSHMveVFVdz09    엄마 선물로 사드렸는데 \n보습력도 좋고 향도 만족한다며 좋아하시네요^^ \n가격도...
K09aUDdkaDQwMjBJeTBUTG8rdnBnQT09    트러블이 자주 나는 피부인 친구에게 선물하려고 샀어요 올인원제품이라서 사용이 편할거같네요
K0E1dFFFTFc4UjNlZjF1S2VqRVRIUT09    항상 여드름 스팟용만 사고 \n스킨 로션은 아무거나 사서 썼는데 후회 했어요. \n...
K0FzNHYwcUhlLzIxUHRUQm9xVitTdz09    자극없이 순하고 복합성 피부가 무난하게 사용하기 좋습니다. \n\n여름철까지 잘 쓰...
                                                          ...                        
enVPcUF6Zmh6Ylovc2lkTFVHdmZ1UT09    로션은 처음 구매해보는데 좋은 것같아요! 제형은 생각보다 묽어서 보습감이 부족할 줄...
enZUZVl2dnREd2RONUEzWVdha2RzZz09                     닦는용도로 쓰고 있으며 향은 딱히 나지않았던거 같아요~^^
enc4V2xrWG1lVG54emxKU3E3dzRyUT09    촉촉하고 발림성이 좋아요~ 끈적임 없어서 사용하기 불편하지 않습니다! 무난하게 남자...
enlJL3NnSHZLNmx1QyswNGlmT05rQT09    이거 성분도 좋고 피부장벽을 튼튼하게 만들어주는거 같아요! 단 지성이 쓰기엔 보습이...
enlwL2hrSTZSQitLMWVLcHY0MDdhQT09    일단 하루 써보았는데 만족하구요. 기존에 다른제품 쓰던걸 바꿔보려는건데 좀더 써보고...
Name: review, Length: 3576, dtype: object

In [30]:
user = user.join(user_review_agg, on='uid')
user = user.loc[~user.review.isna()]

uid2num = {}
num2uid = {}
for i, uid in enumerate(user.uid):
	uid2num[uid] = i
	num2uid[i] = uid

user.uid = user.uid.apply(lambda x : uid2num[x])
purchase.uid = purchase.uid.apply(lambda x: uid2num[x])

In [31]:
purchase  = purchase.drop('review', axis =1)
purchase = purchase.reset_index().drop('index', axis =1)
product = product.drop('index', axis =1)
user = user.reset_index().drop('index', axis =1)
user = user.drop('uid', axis =1)

product.to_csv('data/processed_product.csv')
purchase.to_csv('data/processed_purchase.csv')
user.to_csv('data/processed_user.csv')

In [32]:
pd.Series(num2uid).to_csv('data/dict_num2uid.csv')
pd.Series(num2pid).to_csv('data/dict_num2pid.csv')
pd.Series(num2category).to_csv('data/dict_num2category.csv')
pd.Series(num2spec).to_csv('data/dict_num2spec.csv')
pd.Series(num2ingredient).to_csv('data/dict_num2ingredient.csv')
pd.Series(num2brand).to_csv('data/dict_num2brand.csv')